In [409]:
import sys

if 'sympy' not in sys.modules:
  assert('Install sympy with - pip install sympy')

from sympy import Matrix, Symbol, sqrt, solve, Eq, expand, pi, print_latex, latex
from IPython.display import Markdown as md
from IPython.core.interactiveshell import InteractiveShell as shell
shell.ast_node_interactivity = 'all'

def crossP(vec1, vec2):
  return vec1.col(0).cross(vec2)

# generate cross product as latex
def crossPL(vec1, vec2):
  return f'${latex(expand(vec1))} \\times {latex(expand(vec2))}$'

def crossPLs(vec1, vec2):
  return f'${latex(expand(vec1.col(0).cross(vec2)))}$'

## Angabe des Systems
![alternatvie text](src/system.png)

## Angabe der Kräfte

$$ F_C = \sqrt{2}LC $$
$$ qV_1(s_1) = q $$
$$ qV_3(s_3) = \frac{qx_3}{L} $$
$$ qz_4(s_4) = q $$

In [410]:
# DEFINE ALL VARIABLES
q = Symbol('q')
L = Symbol('L')
Ay = Symbol('A_y')
Cx = Symbol('C_x')
Cy = Symbol('C_y')
Dx = Symbol('D_x')
Ex = Symbol('E_x')
Fx = Symbol('F_x')
MA = Symbol('M_A')

In [411]:
# SETUP ALL VECTORS
R_1 = Matrix([sqrt(2)/2 * 3*pi*L*q/4, -sqrt(2)/2 * 3*pi*L*q/4,0])
R_2 = Matrix([0, -sqrt(3)*L/2*q, 0])
R_3 = Matrix([0,-sqrt(3)*q/4,0])
x_R3 = Matrix([L/sqrt(3),0,0])

F_c = Matrix([q*L, q*L,0])

A = Matrix([0,Ay,0])
C = Matrix([-Cx,-Cy, 0])
D = Matrix([Dx,0,0])
E = Matrix([Ex,0,0])
F = Matrix([Fx,0,0])
M_A = Matrix([0,0,MA])

R_1
R_2
R_3

sqrt(R_1[0]**2 + R_1[1]**2)

Matrix([
[ 3*sqrt(2)*pi*L*q/8],
[-3*sqrt(2)*pi*L*q/8],
[                  0]])

Matrix([
[             0],
[-sqrt(3)*L*q/2],
[             0]])

Matrix([
[           0],
[-sqrt(3)*q/4],
[           0]])

3*pi*sqrt(L**2*q**2)/4

-----
### Berechnung von Auflager $D_x$

In [412]:
x_Dx = Matrix([0,L/2,0])
tmp = (crossP(x_R3, R_3) + crossP(x_Dx, D)).row(2)

D[0] = solve(tmp, Dx).popitem()[1]
D

Matrix([
[-q/2],
[   0],
[   0]])

-----
### Berechnung von Auflager $C$
#### 1. $C_y$

In [413]:
t = solve(C[1]+F_c[1]+R_3[1], Cy)
C[1] = -expand(t[0])
C

Matrix([
[              -C_x],
[-L*q + sqrt(3)*q/4],
[                 0]])

#### 2. $C_x$

In [414]:
x_Dx = Matrix([0, L, 0])
x_R3 = Matrix([-L/(2*sqrt(3)),0,0])
d_Fc = Matrix([-sqrt(3)*L/2, L/2, 0])
d_C  = Matrix([-sqrt(3)*L/2, L/2, 0])

tmp = (crossP(x_Dx, D)+crossP(x_R3, R_3)+crossP(d_Fc, F_c)+crossP(d_C, C)).row(2)
C[0] = -solve(tmp, Cx).popitem()[1]

C


Matrix([
[        -L*q + q/2],
[-L*q + sqrt(3)*q/4],
[                 0]])

----
### Berechnung von Auflager $A$
#### 1. $A_x$
$ +\uparrow \sum{F_y} \stackrel{!}{=} 0 $

In [415]:
tmp = A[1]+R_1[1]+R_2[1]+F_c[1]+R_3[1]
A[1] = expand(solve(tmp, Ay)[0])
A

Matrix([
[                                                      0],
[-L*q + sqrt(3)*L*q/2 + 3*sqrt(2)*pi*L*q/8 + sqrt(3)*q/4],
[                                                      0]])

#### 2. $M_A$
$ \stackrel{\curvearrowleft}{+} \sum{M_G} \stackrel{!}{=} 0 $

In [416]:
d_A  = Matrix([-sqrt(3)*L/2, (L+L/2+L/4), 0])
d_R2 = Matrix([-sqrt(3)*L/4, (L+L/2+L/4), 0])
d_C  = Matrix([L, (L+L/2), 0])
d_Fc = Matrix([L, (L+L/2), 0])

tmp = crossP(d_R2, R_2) + crossP(d_A, A) + crossP(d_Fc, F_c) + crossP(d_C, C*(-1)) + M_A
md(f'{crossPL(d_R2, R_2)} + {crossPL(d_A, A)} + {crossPL(d_Fc, F_c)} + {crossPL(d_C, C*(-1))} + ${latex(M_A)}$')
md(f'{crossPLs(d_R2, R_2)} + {crossPLs(d_A, A)} + {crossPLs(d_Fc, F_c)} + {crossPLs(d_C, C*(-1))} + ${latex(M_A)}$')
# M_A[2] = solve(tmp, MA).popitem()[1]
hmm_1 = expand(solve(tmp, MA).popitem()[1])

$\left[\begin{matrix}- \frac{\sqrt{3} L}{4}\\\frac{7 L}{4}\\0\end{matrix}\right] \times \left[\begin{matrix}0\\- \frac{\sqrt{3} L q}{2}\\0\end{matrix}\right]$ + $\left[\begin{matrix}- \frac{\sqrt{3} L}{2}\\\frac{7 L}{4}\\0\end{matrix}\right] \times \left[\begin{matrix}0\\- L q + \frac{\sqrt{3} L q}{2} + \frac{3 \sqrt{2} \pi L q}{8} + \frac{\sqrt{3} q}{4}\\0\end{matrix}\right]$ + $\left[\begin{matrix}L\\\frac{3 L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}L q\\L q\\0\end{matrix}\right]$ + $\left[\begin{matrix}L\\\frac{3 L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}L q - \frac{q}{2}\\L q - \frac{\sqrt{3} q}{4}\\0\end{matrix}\right]$ + $\left[\begin{matrix}0\\0\\M_{A}\end{matrix}\right]$

$\left[\begin{matrix}0\\0\\\frac{3 L^{2} q}{8}\end{matrix}\right]$ + $\left[\begin{matrix}0\\0\\- \frac{3 \sqrt{6} \pi L^{2} q}{16} - \frac{3 L^{2} q}{4} + \frac{\sqrt{3} L^{2} q}{2} - \frac{3 L q}{8}\end{matrix}\right]$ + $\left[\begin{matrix}0\\0\\- \frac{L^{2} q}{2}\end{matrix}\right]$ + $\left[\begin{matrix}0\\0\\- \frac{L^{2} q}{2} - \frac{\sqrt{3} L q}{4} + \frac{3 L q}{4}\end{matrix}\right]$ + $\left[\begin{matrix}0\\0\\M_{A}\end{matrix}\right]$

-----
### Berechnung von Auflager $F_x$
$ \stackrel{+}{\rightarrow} \sum{F_x} \stackrel{!}{=} 0 $

$ C_x + F_x + D_x + F_x = 0 $

In [417]:
tmp = C[0]+F_c[0]+D[0]+F[0]
F[0] = solve(tmp, Fx)
F

Matrix([
[0],
[0],
[0]])

$ \stackrel{\curvearrowleft}{+} \sum{M_D} = 0 $

In [418]:
d_R3 = Matrix([-(sqrt(3)*L/(2*3)), -L/4, 0])
d_C = Matrix([-sqrt(3)*L/2, -L/2, 0])
d_Fc = Matrix([-sqrt(3)*L/2, -L/2, 0])
d_F = Matrix([0, -L, 0])

tmp = crossP(d_R3, R_3) + crossP(d_C, C) + crossP(d_Fc, F_c) + crossP(d_F, Matrix([Fx, 0, 0]))
# F[0] = solve(tmp[2], Fx)[0]
solve(tmp[2], Fx)[0]
# tmp

0

----
### Berechnung von Auflager $E_x$
$ \stackrel{+}{\rightarrow} \sum{F_x} \stackrel{!}{=} 0 $

In [419]:
tmp = D[0]+F[0]+F_c[0]+R_1[0]+E[0]
E[0] = expand(solve(tmp, Ex)[0])
E

Matrix([
[-3*sqrt(2)*pi*L*q/8 - L*q + q/2],
[                              0],
[                              0]])

----
### Kontrolle durch Summe der Momente um Gelenk $C$
$ \stackrel{\curvearrowleft}{+} \sum{M_C} = 0 $ im Gesamtsystem

In [420]:
# d_R1 = Matrix([-(3*L/2 - 3*L/2*1/sqrt(2)),-(L+3*L/2*1/sqrt(2)),0])
d_R1 = Matrix([-(3*L/2*sqrt(2)),-(3*L/2 - 3*L/2*sqrt(2)),0])
d_R2 = Matrix([-(L+sqrt(3)*L/4),L/4,0])
d_R3 = Matrix([L/sqrt(3),0,0])

d_A = Matrix([-(L+3*L/2), -3*L/2, 0])
d_D = Matrix([sqrt(3)*L/2, L/2,0])
d_E = Matrix([-(L+sqrt(3)*L/2),L/2,0])
d_F = Matrix([sqrt(3)*L/2, -L/2,0])

tmp = crossP(d_A, A) + crossP(d_D,D) + crossP(d_E, E) + crossP(d_F, F) + crossP(d_R1, R_1) + crossP(d_R2, R_2) + crossP(d_R3, R_3)
md(f'{crossPL(d_A, A)} + {crossPL(d_D,D) } + {crossPL(d_E, E)} + {crossPL(d_F, F)} + {crossPL(d_R1, R_1)} + {crossPL(d_R2, R_2)} + {crossPL(d_R3, R_3)}' )
expand(tmp[2])

R_1

$\left[\begin{matrix}- \frac{5 L}{2}\\- \frac{3 L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}0\\- L q + \frac{\sqrt{3} L q}{2} + \frac{3 \sqrt{2} \pi L q}{8} + \frac{\sqrt{3} q}{4}\\0\end{matrix}\right]$ + $\left[\begin{matrix}\frac{\sqrt{3} L}{2}\\\frac{L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}- \frac{q}{2}\\0\\0\end{matrix}\right]$ + $\left[\begin{matrix}- L - \frac{\sqrt{3} L}{2}\\\frac{L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}- \frac{3 \sqrt{2} \pi L q}{8} - L q + \frac{q}{2}\\0\\0\end{matrix}\right]$ + $\left[\begin{matrix}\frac{\sqrt{3} L}{2}\\- \frac{L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}0\\0\\0\end{matrix}\right]$ + $\left[\begin{matrix}- \frac{3 \sqrt{2} L}{2}\\- \frac{3 L}{2} + \frac{3 \sqrt{2} L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}\frac{3 \sqrt{2} \pi L q}{8}\\- \frac{3 \sqrt{2} \pi L q}{8}\\0\end{matrix}\right]$ + $\left[\begin{matrix}- L - \frac{\sqrt{3} L}{4}\\\frac{L}{4}\\0\end{matrix}\right] \times \left[\begin{matrix}0\\- \frac{\sqrt{3} L q}{2}\\0\end{matrix}\right]$ + $\left[\begin{matrix}\frac{\sqrt{3} L}{3}\\0\\0\end{matrix}\right] \times \left[\begin{matrix}0\\- \frac{\sqrt{3} q}{4}\\0\end{matrix}\right]$

-3*sqrt(3)*L**2*q/4 - 3*sqrt(2)*pi*L**2*q/16 + 27*L**2*q/8 - 5*sqrt(3)*L*q/8 - L*q/4

Matrix([
[ 3*sqrt(2)*pi*L*q/8],
[-3*sqrt(2)*pi*L*q/8],
[                  0]])


$ \stackrel{\curvearrowleft}{+} \sum{M_C} = 0 $ in Freischnitt I

In [421]:
# d_R1 = Matrix([-(3*L/2 - 3*L/2*1/sqrt(2)),-(L+3*L/2*1/sqrt(2)),0])
d_R2 = Matrix([-(L+sqrt(3)*L/4),L/4,0])
d_R3 = Matrix([L/sqrt(3),0,0])

d_A = Matrix([-(L+3*L/2), -3*L/2, 0])
d_D = Matrix([sqrt(3)*L/2, L/2,0])
d_E = Matrix([-(L+sqrt(3)*L/2),L/2,0])
d_F = Matrix([sqrt(3)*L/2, -L/2,0])

tmp = crossP(d_A, A) + crossP(d_E, E) + crossP(d_R1, R_1) + crossP(d_R2, R_2)
md(f'{crossPL(d_A, A)} + {crossPL(d_E, E)} + {crossPL(d_R1, R_1)} + {crossPL(d_R2, R_2)}')
md(f'{crossPLs(d_A, A)} + {crossPLs(d_E, E)} + {crossPLs(d_R1, R_1)} + {crossPLs(d_R2, R_2)}')
expand(tmp[2])


$\left[\begin{matrix}- \frac{5 L}{2}\\- \frac{3 L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}0\\- L q + \frac{\sqrt{3} L q}{2} + \frac{3 \sqrt{2} \pi L q}{8} + \frac{\sqrt{3} q}{4}\\0\end{matrix}\right]$ + $\left[\begin{matrix}- L - \frac{\sqrt{3} L}{2}\\\frac{L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}- \frac{3 \sqrt{2} \pi L q}{8} - L q + \frac{q}{2}\\0\\0\end{matrix}\right]$ + $\left[\begin{matrix}- \frac{3 \sqrt{2} L}{2}\\- \frac{3 L}{2} + \frac{3 \sqrt{2} L}{2}\\0\end{matrix}\right] \times \left[\begin{matrix}\frac{3 \sqrt{2} \pi L q}{8}\\- \frac{3 \sqrt{2} \pi L q}{8}\\0\end{matrix}\right]$ + $\left[\begin{matrix}- L - \frac{\sqrt{3} L}{4}\\\frac{L}{4}\\0\end{matrix}\right] \times \left[\begin{matrix}0\\- \frac{\sqrt{3} L q}{2}\\0\end{matrix}\right]$

$\left[\begin{matrix}0\\0\\- \frac{15 \sqrt{2} \pi L^{2} q}{16} - \frac{5 \sqrt{3} L^{2} q}{4} + \frac{5 L^{2} q}{2} - \frac{5 \sqrt{3} L q}{8}\end{matrix}\right]$ + $\left[\begin{matrix}0\\0\\\frac{L^{2} q}{2} + \frac{3 \sqrt{2} \pi L^{2} q}{16} - \frac{L q}{4}\end{matrix}\right]$ + $\left[\begin{matrix}0\\0\\\frac{9 \sqrt{2} \pi L^{2} q}{16}\end{matrix}\right]$ + $\left[\begin{matrix}0\\0\\\frac{3 L^{2} q}{8} + \frac{\sqrt{3} L^{2} q}{2}\end{matrix}\right]$

-3*sqrt(3)*L**2*q/4 - 3*sqrt(2)*pi*L**2*q/16 + 27*L**2*q/8 - 5*sqrt(3)*L*q/8 - L*q/4

In [422]:
hmm_1
hmm_2
expand(tmp[2])

-sqrt(3)*L**2*q/2 + 11*L**2*q/8 + 3*sqrt(6)*pi*L**2*q/16 - 3*L*q/8 + sqrt(3)*L*q/4

-sqrt(3)*L**2*q/2 + 7*L**2*q/8 + 3*sqrt(6)*pi*L**2*q/16 - 3*L*q/8 + sqrt(3)*L*q/4

-3*sqrt(3)*L**2*q/4 - 3*sqrt(2)*pi*L**2*q/16 + 27*L**2*q/8 - 5*sqrt(3)*L*q/8 - L*q/4